In [4]:
%matplotlib inline
import matplotlib.pyplot as plt

In [108]:
import os
import glob
import shutil
import numpy as np

import cv2
import skimage
from skimage.morphology import remove_small_objects, remove_small_holes, binary_erosion, binary_dilation

output_folder = 'stage1_test_out'
sub_folder = 'stage1_test_sub'

In [88]:
for mask in glob.glob(output_folder + '/*_mask.png'):
    os.makedirs(output_folder + '_masks', exist_ok=True)
    shutil.move(mask, output_folder + '_masks')

In [127]:
def rle_encoding(x):
    dots = np.where(x.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b > prev + 1):
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def get_rle(image):
    for i in range(1, image.max() + 1):
        yield rle_encoding(image == i)

In [ ]:
image_ids = []
rles = []

for i, file in enumerate(os.listdir(output_folder)):
    img_id = file[:-4]

    image_path = os.path.join(output_folder, img_id + '.png')
    masks_path = os.path.join(output_folder + '_masks', img_id + '_mask.png')

    depth_image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    masks_image = cv2.imread(masks_path, cv2.IMREAD_UNCHANGED)

    masks_image = remove_small_objects((masks_image > 0), min_size=32)    
    masks_image = binary_erosion(masks_image, selem=np.ones((5, 5)))
    masks_image = binary_dilation(masks_image, selem=np.ones((7, 7)))
    masks_image = remove_small_holes(masks_image, min_size=128)
    
    result_image = np.zeros(shape=depth_image.shape, dtype=np.float32)

    cut_image = remove_small_objects((depth_image > 0), min_size=32)
    cut_image = remove_small_holes(cut_image, min_size=128)
    
    cut_image = cut_image * masks_image
    
    cut_labels = skimage.morphology.label(cut_image)
    
    cut_ids = np.unique(cut_labels)[1:]
    for instance_id in cut_ids:
        instance_mask = binary_dilation((cut_labels == instance_id), np.ones((3, 3)))
        instance_mask = remove_small_holes(instance_mask, min_size=32)
        result_image[instance_mask] = instance_id

    result_image = result_image.astype(np.uint16)
    
    #plt.figure(figsize=(8, 8))
    #plt.imshow(cut_image, cmap='gray')
    #plt.show()
    
    for rle in get_rle(result_image):
        image_ids.append(img_id)
        rles.append(rle)

    if not result_image.any():
        print('wtf')
        
    print(i)
        
    #cv2.imwrite(os.path.join(sub_folder, img_id + '.png'), result_image)

In [ ]:
print(len(os.listdir(output_folder)))

In [176]:
import pandas as pd

submission = pd.DataFrame(data={'ImageId': image_ids,
                                'EncodedPixels': [' '.join(map(str, x)) for x in rles]})

In [177]:
submission.to_csv('./submission.csv', index=None)